In [ ]:
!pip install qiskit==0.41.0 qiskit-aer==0.11.2 qiskit-optimization==0.4.0


In [ ]:
import heapq
from datetime import datetime, timedelta
from qiskit import QuantumCircuit, Aer, execute
import random


class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        """
        Initialize the system with stops and graph data.
        :param stops: Dictionary of bus stops with details.
        :param adjacency_list: Graph adjacency list with distances and times.
        """
        self.stops = stops
        self.graph = adjacency_list

    def phase_estimation(self, distance, time):
        """
        Use Quantum Phase Estimation (QPE) to encode the cost.
        :param distance: Distance between two stops.
        :param time: Time between two stops.
        :return: Measured phase result encoding composite metric.
        """
        qc = QuantumCircuit(6, 6)
        qc.h(range(6))
        composite_metric = distance + 0.1 * time

        for i in range(6):
            qc.p(composite_metric / (2 ** i), i)  # Use phase (p) gate
        qc.measure(range(6), range(6))

        backend = Aer.get_backend('qasm_simulator')
        result = execute(qc, backend, shots=1024).result()
        counts = result.get_counts()
        return min(counts, key=counts.get)  # Return the minimum phase

    def modified_dijkstra(self, source, destinations, start_time):
        """
        Find the optimal route using Modified Dijkstra's with Quantum Phase Estimation.
        :param source: Starting stop.
        :param destinations: List of required stops.
        :param start_time: Starting time (datetime object).
        :return: Tuple (total_cost, route, end_time).
        """
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if all(stop in path for stop in destinations):
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                # Use Quantum Phase Estimation to calculate composite metric
                quantum_metric = int(self.phase_estimation(distance, time), 2)

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None  # No feasible route



# Define source, destinations, and start time
source = "Node_1"
destinations = [f"Node_47"]
start_time = datetime.now()

# Initialize and execute Quantum Modified Dijkstra
quantum_dijkstra = QuantumModifiedDijkstra(nodes, graph)
cost, route, end_time = quantum_dijkstra.modified_dijkstra(source, destinations, start_time)

# Print results
print(f"Total Cost: {cost}")
print(f"Route: {route}")
print(f"End Time: {end_time}")


Total Cost: 161
Route: ['Node_1', 'Node_45', 'Node_43', 'Node_8', 'Node_28', 'Node_36', 'Node_23', 'Node_47']
End Time: 2024-12-22 15:41:02.230591


In [ ]:
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your IBM Quantum API token (do this only once)
# Uncomment the line below if running for the first time
QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN")

# Load the IBM Quantum service
service = QiskitRuntimeService()

# Set up the backend
backend = service.backend("ibmq_qasm_simulator")  # Free-tier simulator

source = "Node_1"
destination = "Node_45"

def create_qubo(graph, source, destination):
    qubo = QuadraticProgram()

    # Add variables for each node
    for node in graph:
        qubo.binary_var(f"x_{node}")

    # Add objective function: Minimize distance
    quadratic_terms = {}
    for node, neighbors in graph.items():
        for neighbor, details in neighbors.items():
            quadratic_terms[(f"x_{node}", f"x_{neighbor}")] = details["distance"]

    qubo.minimize(quadratic=quadratic_terms)

    # Add constraints: Source and destination must be part of the path
    qubo.linear_constraint({f"x_{source}": 1}, "==", 1, "source_constraint")
    qubo.linear_constraint({f"x_{destination}": 1}, "==", 1, "destination_constraint")

    return qubo

def solve_with_qaoa(qubo):
    # Convert constraints to penalty terms
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qubo)

    # Define QAOA instance
    optimizer = COBYLA(maxiter=100)
    qaoa = QAOA(optimizer=optimizer, reps=1, quantum_instance=backend)

    # Solve with Minimum Eigen Optimizer
    qaoa_optimizer = MinimumEigenOptimizer(qaoa)
    result = qaoa_optimizer.solve(qubo)

    return result

# Example graph (replace with your actual graph)
graph = {
    "Node_1": {"Node_2": {"distance": 5}, "Node_3": {"distance": 10}},
    "Node_2": {"Node_3": {"distance": 2}, "Node_45": {"distance": 8}},
    "Node_3": {"Node_45": {"distance": 7}},
    "Node_45": {}
}

# Create QUBO and solve
qubo = create_qubo(graph, source, destination)
result = solve_with_qaoa(qubo)

# Extract the optimal path
selected_nodes = [var.name for var, value in zip(qubo.variables, result.x) if value > 0.5]

print("\nOptimal Path:")
print(selected_nodes)


ModuleNotFoundError: No module named 'qiskit.algorithms'

In [ ]:
import numpy as np
from scipy.optimize import minimize
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

# Define the QUBO problem

def create_qubo(graph, source, destination):
    qubo = QuadraticProgram()

    # Add variables for each node
    for node in graph:
        qubo.binary_var(f"x_{node}")

    # Add the objective function (minimize distance)
    quadratic_terms = {}
    for node, neighbors in graph.items():
        for neighbor, details in neighbors.items():
            quadratic_terms[(f"x_{node}", f"x_{neighbor}")] = details['distance']

    qubo.minimize(quadratic=quadratic_terms)

    # Add constraints to include the source and destination nodes
    qubo.linear_constraint({f"x_{source}": 1}, "==", 1, f"source_constraint_{source}")
    qubo.linear_constraint({f"x_{destination}": 1}, "==", 1, f"destination_constraint_{destination}")

    return qubo

# Classical QUBO Solver using Scipy

def classical_qubo_solver(qubo, graph):
    # Convert QUBO to a matrix format
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qubo)

    num_vars = len(qubo.variables)
    qubo_matrix = np.zeros((num_vars, num_vars))

    # Fill the QUBO matrix from quadratic terms
    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        qubo_matrix[i, j] = value
        qubo_matrix[j, i] = value  # Ensure symmetry

    # Classical solver for QUBO
    def objective(x):
        return x @ qubo_matrix @ x.T

    # Initial guess for binary variables
    initial_guess = np.random.choice([0, 1], size=num_vars)

    # Solve using Scipy minimize
    result = minimize(
        objective,
        initial_guess,
        bounds=[(0, 1)] * num_vars,
        method="L-BFGS-B",
        options={"disp": True}
    )

    # Round to nearest binary values
    solution = np.round(result.x)

    # Calculate the cost based on the graph
    total_cost = 0
    for i, node_i in enumerate(qubo.variables):
        if solution[i] > 0.5:
            for j, node_j in enumerate(qubo.variables):
                if solution[j] > 0.5 and (f"x_{node_i.name}", f"x_{node_j.name}") in qubo.objective.quadratic.to_dict():
                    total_cost += graph[node_i.name[2:]][node_j.name[2:]]['distance']

    return solution, total_cost

# Map solution to node names

def extract_path_from_solution(qubo, solution):
    path_nodes = [var.name[2:] for var, value in zip(qubo.variables, solution) if value > 0.5]
    return path_nodes

# Generate the QUBO for the graph
source = "Node_1"
destination = "Node_47"

qubo = create_qubo(graph, source, destination)

# Solve the QUBO problem classically
solution, cost = classical_qubo_solver(qubo, graph)
optimal_path = extract_path_from_solution(qubo, solution)

# Print results
print("Optimal Path:")
print(optimal_path)
print(f"Optimal Cost: {cost}")


Optimal Path:
['Node_28', 'Node_35', 'Node_44', 'Node_49']
Optimal Cost: 0


In [ ]:
import random
import heapq
import numpy as np
from scipy.optimize import minimize
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

# Dynamic Graph Simulation
class DynamicGraph:
    def __init__(self, nodes, graph):
        self.nodes = nodes
        self.graph = graph

    def update_graph(self):
        """Simulate real-time updates to the graph."""
        for node, edges in self.graph.items():
            for neighbor in list(edges.keys()):
                if random.random() < 0.1:  # 10% chance to remove an edge
                    del self.graph[node][neighbor]
                else:  # Adjust weights
                    self.graph[node][neighbor]['distance'] *= random.uniform(0.9, 1.1)
                    self.graph[node][neighbor]['time'] *= random.uniform(0.9, 1.1)

    def add_random_edges(self, num_edges=5):
        """Add random edges to the graph."""
        for _ in range(num_edges):
            node1, node2 = random.sample(self.nodes.keys(), 2)
            if node2 not in self.graph[node1]:
                self.graph[node1][node2] = {
                    'distance': random.uniform(1, 50),
                    'time': random.uniform(1, 30)
                }

    def print_graph(self):
        """Print the graph nodes and edges."""
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")



In [ ]:
import random
import heapq
from datetime import datetime, timedelta
from qiskit import QuantumCircuit, Aer, execute


class DynamicGraph:
    def __init__(self, num_nodes, max_edges, max_distance, max_time):
        self.nodes = {
            f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))}
            for i in range(1, num_nodes + 1)
        }
        self.graph = {
            node: {
                neighbor: {
                    "distance": random.uniform(1, max_distance),
                    "time": random.uniform(1, max_time),
                }
                for neighbor in random.sample(list(self.nodes.keys()), random.randint(1, max_edges))
                if neighbor != node
            }
            for node in self.nodes
        }

    def update_graph(self):
        for node, edges in self.graph.items():
            for neighbor in list(edges.keys()):
                if random.random() < 0.1:  # 10% chance to remove an edge
                    del self.graph[node][neighbor]
                else:
                    self.graph[node][neighbor]['distance'] *= random.uniform(0.9, 1.1)
                    self.graph[node][neighbor]['time'] *= random.uniform(0.9, 1.1)

    def print_graph(self):
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")


class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
        try:
            qc = QuantumCircuit(6, 6)
            qc.h(range(6))
            composite_metric = distance + 0.1 * time

            for i in range(6):
                qc.p(composite_metric / (2 ** i), i)
            qc.measure(range(6), range(6))

            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()

            measured_bitstring = min(counts, key=counts.get)
            return int(measured_bitstring, 2)
        except Exception as e:
            print(f"Error in phase estimation: {e}")
            return float('inf')

    def modified_dijkstra(self, source, destination, start_time):
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                if quantum_metric == float('inf'):
                    continue

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None


# Generate Dynamic Graph
num_nodes = 10
max_edges = 3
max_distance = 50
max_time = 30
dynamic_graph = DynamicGraph(num_nodes, max_edges, max_distance, max_time)

# Print Initial Graph
print("Initial Graph:")
dynamic_graph.print_graph()

# Update Graph
dynamic_graph.update_graph()

# Initialize Quantum Modified Dijkstra
qmd = QuantumModifiedDijkstra(dynamic_graph.nodes, dynamic_graph.graph)

# Define Source and Destination
source = "Node_1"
destination = f"Node_{num_nodes}"
start_time = datetime.now()

# Run Quantum Modified Dijkstra
cost, path, end_time = qmd.modified_dijkstra(source, destination, start_time)

# Print Results
print(f"\nCost: {cost}")
print(f"Path: {path}")
print(f"End Time: {end_time}")


Initial Graph:

Graph Nodes and Edges:
Node: Node_1, Coordinates: (37.26740439965044, 84.99813669121026)
  -> Node_5: Distance=17.68, Time=19.15
Node: Node_2, Coordinates: (78.06815974806534, 84.55807176739928)
  -> Node_1: Distance=12.60, Time=6.34
  -> Node_10: Distance=34.80, Time=3.35
  -> Node_8: Distance=41.96, Time=12.06
Node: Node_3, Coordinates: (3.3625534314995487, 14.448067225621465)
  -> Node_9: Distance=30.37, Time=4.68
  -> Node_6: Distance=5.67, Time=23.42
  -> Node_8: Distance=6.24, Time=22.85
Node: Node_4, Coordinates: (6.284461129206886, 9.634324258348459)
  -> Node_8: Distance=31.04, Time=7.35
  -> Node_3: Distance=26.96, Time=3.91
Node: Node_5, Coordinates: (84.26167421939827, 71.46962146314871)
  -> Node_1: Distance=20.80, Time=19.90
  -> Node_3: Distance=27.77, Time=8.32
Node: Node_6, Coordinates: (66.74783095598028, 91.35636449807882)
  -> Node_10: Distance=24.28, Time=8.22
  -> Node_1: Distance=36.09, Time=10.08
Node: Node_7, Coordinates: (16.14572945034857, 85.

In [ ]:
import random
import heapq
from qiskit import QuantumCircuit, Aer, execute
# Quantum A* Implementation
class QuantumAStar:
    def __init__(self, nodes, adjacency_list):
        self.nodes = nodes
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def quantum_heuristic(self, current, destination):
        """
        Heuristic calculation with Quantum Phase Estimation.
        """
        try:
            qc = QuantumCircuit(6, 6)
            current_coords = self.nodes[current]['coords']
            dest_coords = self.nodes[destination]['coords']
            distance = ((current_coords[0] - dest_coords[0]) ** 2 +
                        (current_coords[1] - dest_coords[1]) ** 2) ** 0.5
            for i in range(6):
                qc.p(distance / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))
            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            heuristic = int(min(counts, key=counts.get), 2)

            # Normalize heuristic to align with distance and time scales
            normalized_heuristic = heuristic / len(self.nodes)
            return normalized_heuristic
        except Exception as e:
            print(f"Error in quantum heuristic: {e}")
            return float('inf')

    def find_optimized_route(self, source, destination):
        queue = [(0, source, [source])]
        visited = set()

        while queue:
            f_cost, current, path = heapq.heappop(queue)

            if current in visited:
                continue
            visited.add(current)

            if current == destination:
                return f_cost, path

            for neighbor, data in self.graph[current].items():
                if neighbor not in visited:
                    distance = data['distance']
                    time = data['time']
                    heuristic = self.quantum_heuristic(neighbor, destination)
                    new_f = f_cost + distance + 0.1 * time + heuristic
                    heapq.heappush(queue, (new_f, neighbor, path + [neighbor]))

        return float('inf'), []

In [ ]:
import heapq
from datetime import datetime, timedelta
from qiskit import QuantumCircuit, Aer, execute
import random


# Quantum Modified Dijkstra Implementation
class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
        """
        Use Quantum Phase Estimation (QPE) to encode the cost.
        """
        try:
            qc = QuantumCircuit(6, 6)
            qc.h(range(6))
            composite_metric = distance + 0.1 * time

            for i in range(6):
                qc.p(composite_metric / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))

            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            quantum_metric = int(min(counts, key=counts.get), 2)

            # Normalize quantum metric
            normalized_metric = quantum_metric / len(self.stops)
            return normalized_metric
        except Exception as e:
            print(f"Error in phase estimation: {e}")
            return float('inf')

    def modified_dijkstra(self, source, destination, start_time):
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                if quantum_metric == float('inf'):
                    continue

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None

In [ ]:
# QAOA-Based Optimization (Simulated)
def create_qubo(graph, source, destination):
    qubo = QuadraticProgram()
    for node in graph:
        qubo.binary_var(f"x_{node}")

    quadratic_terms = {}
    for node, neighbors in graph.items():
        for neighbor, details in neighbors.items():
            quadratic_terms[(f"x_{node}", f"x_{neighbor}")] = details['distance']

    qubo.minimize(quadratic=quadratic_terms)
    qubo.linear_constraint({f"x_{source}": 1}, "==", 1, "source_constraint")
    qubo.linear_constraint({f"x_{destination}": 1}, "==", 1, "destination_constraint")

    return qubo

def solve_qubo_classically(qubo, graph):
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qubo)

    num_vars = len(qubo.variables)
    qubo_matrix = np.zeros((num_vars, num_vars))

    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        qubo_matrix[i, j] = value
        qubo_matrix[j, i] = value

    def objective(x):
        return x @ qubo_matrix @ x.T

    initial_guess = np.random.choice([0, 1], size=num_vars)
    result = minimize(objective, initial_guess, bounds=[(0, 1)] * num_vars, method="L-BFGS-B")
    solution = np.round(result.x).astype(int)

    # Calculate the cost based on the solution
    total_cost = 0
    path_edges = []
    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        if solution[i] > 0.5 and solution[j] > 0.5:
            node_i = qubo.variables[i].name.split("_")[1]
            node_j = qubo.variables[j].name.split("_")[1]
            if node_j in graph[node_i]:  # Ensure the edge exists in the graph
                total_cost += graph[node_i][node_j]['distance']
                path_edges.append((node_i, node_j))

    return solution, total_cost, path_edges


# Simulation Setup
simulation_nodes = {f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))} for i in range(1, 151)}
simulation_graph = {node: {
    neighbor: {"distance": random.uniform(1, 50), "time": random.uniform(1, 30)}
    for neighbor in random.sample(list(simulation_nodes.keys()), random.randint(1, 5))
    if neighbor != node
} for node in simulation_nodes}

dynamic_graph = DynamicGraph(simulation_nodes, simulation_graph)





In [ ]:
# Define source and destination
source = "Node_1"
destination = "Node_45"
# Initialize Algorithms
astar = QuantumAStar(simulation_nodes, simulation_graph)
dijkstra = QuantumModifiedDijkstra(simulation_nodes, simulation_graph)

# Print Initial Graph
dynamic_graph.print_graph()

# Run Simulation
for iteration in range(5):
    print(f"\nIteration {iteration + 1}:")
    dynamic_graph.update_graph()

    # Print Updated Graph
    dynamic_graph.print_graph()

    # Quantum A*
    cost_astar, path_astar = astar.find_optimized_route(source, destination)
    print(f"Quantum A*: Cost={cost_astar:.2f}, Path={path_astar}")

    # Quantum Modified Dijkstra
# Initialize Quantum Modified Dijkstra
qmd = QuantumModifiedDijkstra(nodes, graph)

# Define Source and Destination
source = "Node_1"
destinations = ["Node_45"]
start_time = datetime.now()

# Run Algorithm
cost, path, end_time = qmd.modified_dijkstra(source, destinations, start_time)

# Output Results
print(f"Cost: {cost}")
print(f"Path: {path}")
print(f"End Time: {end_time}")




Graph Nodes and Edges:
Node: Node_1, Coordinates: (52.22463482069987, 46.42598728164177)
  -> Node_128: Distance=38.47, Time=18.64
  -> Node_37: Distance=7.18, Time=22.82
Node: Node_2, Coordinates: (98.16440472432679, 42.65594913472861)
  -> Node_103: Distance=43.48, Time=6.35
Node: Node_3, Coordinates: (67.40192534977868, 67.43868511323508)
  -> Node_29: Distance=19.14, Time=17.63
Node: Node_4, Coordinates: (25.29865635673172, 56.90531489378833)
  -> Node_34: Distance=38.30, Time=19.28
  -> Node_13: Distance=20.85, Time=23.50
  -> Node_73: Distance=31.46, Time=28.48
Node: Node_5, Coordinates: (40.57278522043085, 11.881419910847868)
  -> Node_97: Distance=44.45, Time=9.70
  -> Node_49: Distance=35.50, Time=20.39
  -> Node_11: Distance=10.14, Time=30.76
  -> Node_74: Distance=5.65, Time=11.09
Node: Node_6, Coordinates: (43.3153248740048, 29.974450447422953)
  -> Node_84: Distance=25.70, Time=2.75
  -> Node_111: Distance=13.81, Time=29.32
  -> Node_70: Distance=22.08, Time=23.16
  -> No

NameError: name 'nodes' is not defined

In [ ]:
    # QAOA (Simulated)
    qubo = create_qubo(simulation_graph, source, destination)
    solution, qaoa_cost, qaoa_path_edges = solve_qubo_classically(qubo, simulation_graph)
    qaoa_path = [f"{edge[0]} -> {edge[1]}" for edge in qaoa_path_edges]
    print(f"QAOA (Simulated): Cost={qaoa_cost:.2f}, Path={qaoa_path}")


QAOA (Simulated): Cost=0.00, Path=[]


In [ ]:
import random
import heapq
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit import QuantumCircuit, Aer, execute


# Dynamic Graph Simulation
class DynamicGraph:
    def __init__(self, num_nodes, max_edges, max_distance, max_time):
        self.nodes = {
            f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))}
            for i in range(1, num_nodes + 1)
        }
        self.graph = {
            node: {
                neighbor: {
                    "distance": random.uniform(1, max_distance),
                    "time": random.uniform(1, max_time),
                }
                for neighbor in random.sample(list(self.nodes.keys()), random.randint(1, max_edges))
                if neighbor != node
            }
            for node in self.nodes
        }

    def update_graph(self):
        for node, edges in self.graph.items():
            for neighbor in list(edges.keys()):
                if random.random() < 0.1:  # 10% chance to remove an edge
                    del self.graph[node][neighbor]
                else:
                    self.graph[node][neighbor]['distance'] *= random.uniform(0.9, 1.1)
                    self.graph[node][neighbor]['time'] *= random.uniform(0.9, 1.1)

    def print_graph(self):
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")


# Quantum Modified Dijkstra
class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
        try:
            qc = QuantumCircuit(6, 6)
            qc.h(range(6))
            composite_metric = distance + 0.1 * time

            for i in range(6):
                qc.p(composite_metric / (2 ** i), i)
            qc.measure(range(6), range(6))

            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            measured_bitstring = min(counts, key=counts.get)
            return int(measured_bitstring, 2)
        except Exception as e:
            print(f"Error in phase estimation: {e}")
            return float('inf')

    def modified_dijkstra(self, source, destination, start_time):
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                if quantum_metric == float('inf'):
                    continue

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None


# QAOA-Based Optimization
def create_qubo(graph, source, destination):
    qubo = QuadraticProgram()
    for node in graph:
        qubo.binary_var(f"x_{node}")

    quadratic_terms = {}
    for node, neighbors in graph.items():
        for neighbor, details in neighbors.items():
            quadratic_terms[(f"x_{node}", f"x_{neighbor}")] = details['distance']

    qubo.minimize(quadratic=quadratic_terms)
    qubo.linear_constraint({f"x_{source}": 1}, "==", 1, "source_constraint")
    qubo.linear_constraint({f"x_{destination}": 1}, "==", 1, "destination_constraint")

    return qubo


def solve_qubo_classically(qubo, graph):
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qubo)

    num_vars = len(qubo.variables)
    qubo_matrix = np.zeros((num_vars, num_vars))

    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        qubo_matrix[i, j] = value
        qubo_matrix[j, i] = value

    def objective(x):
        return x @ qubo_matrix @ x.T

    initial_guess = np.random.choice([0, 1], size=num_vars)
    result = minimize(objective, initial_guess, bounds=[(0, 1)] * num_vars, method="L-BFGS-B")
    solution = np.round(result.x).astype(int)

    total_cost = 0
    path_edges = []
    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        if solution[i] > 0.5 and solution[j] > 0.5:
            node_i = qubo.variables[i].name.split("_")[1]
            node_j = qubo.variables[j].name.split("_")[1]
            if node_j in graph[node_i]:  # Ensure the edge exists in the graph
                total_cost += graph[node_i][node_j]['distance']
                path_edges.append((node_i, node_j))

    return solution, total_cost, path_edges


# Simulation Setup
num_nodes = 10
max_edges = 3
max_distance = 50
max_time = 30
dynamic_graph = DynamicGraph(num_nodes, max_edges, max_distance, max_time)

# Print Initial Graph
dynamic_graph.print_graph()

# Quantum Modified Dijkstra
qmd = QuantumModifiedDijkstra(dynamic_graph.nodes, dynamic_graph.graph)
source = "Node_1"
destination = f"Node_{num_nodes}"
start_time = datetime.now()
cost_dijkstra, path_dijkstra, end_time_dijkstra = qmd.modified_dijkstra(source, destination, start_time)

print("\nQuantum Modified Dijkstra Results:")
print(f"Cost: {cost_dijkstra}")
print(f"Path: {path_dijkstra}")
print(f"End Time: {end_time_dijkstra}")

# QAOA-Based Optimization
qubo = create_qubo(dynamic_graph.graph, source, destination)
solution, total_cost, path_edges = solve_qubo_classically(qubo, dynamic_graph.graph)

print("\nQAOA Results:")
print(f"Solution: {solution}")
print(f"Total Cost: {total_cost}")
print(f"Path Edges: {path_edges}")



Graph Nodes and Edges:
Node: Node_1, Coordinates: (18.70048476062354, 24.449051344592696)
  -> Node_10: Distance=16.14, Time=25.32
Node: Node_2, Coordinates: (52.02480466593718, 10.770710340905909)
  -> Node_4: Distance=8.89, Time=16.50
  -> Node_9: Distance=31.87, Time=4.57
Node: Node_3, Coordinates: (35.24611506459587, 2.3224139775037345)
  -> Node_6: Distance=39.42, Time=1.69
Node: Node_4, Coordinates: (11.870718579440231, 26.046740235477184)
  -> Node_2: Distance=7.72, Time=23.20
  -> Node_7: Distance=3.99, Time=4.16
  -> Node_1: Distance=49.22, Time=24.62
Node: Node_5, Coordinates: (70.9532039170728, 41.373302508827706)
  -> Node_1: Distance=35.45, Time=10.90
Node: Node_6, Coordinates: (84.5483519967799, 49.20315417008363)
  -> Node_2: Distance=42.53, Time=28.43
  -> Node_9: Distance=23.22, Time=14.48
  -> Node_5: Distance=25.86, Time=27.20
Node: Node_7, Coordinates: (82.13945380987724, 61.13447619571532)
  -> Node_1: Distance=28.41, Time=29.21
Node: Node_8, Coordinates: (48.2123

In [ ]:
import random
import heapq
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit import QuantumCircuit, Aer, execute


# Dynamic Graph Simulation
class DynamicGraph:
    def __init__(self, num_nodes, max_edges, max_distance, max_time):
        self.nodes = {
            f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))}
            for i in range(1, num_nodes + 1)
        }
        self.graph = {
            node: {
                neighbor: {
                    "distance": random.uniform(1, max_distance),
                    "time": random.uniform(1, max_time),
                }
                for neighbor in random.sample(list(self.nodes.keys()), random.randint(1, max_edges))
                if neighbor != node
            }
            for node in self.nodes
        }

    def update_graph(self):
        for node, edges in self.graph.items():
            for neighbor in list(edges.keys()):
                if random.random() < 0.1:  # 10% chance to remove an edge
                    del self.graph[node][neighbor]
                else:
                    self.graph[node][neighbor]['distance'] *= random.uniform(0.9, 1.1)
                    self.graph[node][neighbor]['time'] *= random.uniform(0.9, 1.1)

    def print_graph(self):
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")


In [ ]:
# Quantum Modified Dijkstra
class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
      try:
        qc = QuantumCircuit(6, 6)
        qc.h(range(6))
        composite_metric = distance + 0.1 * time

        for i in range(6):
            qc.p(composite_metric / (2 ** i), i)  # Use phase (p) gate
        qc.measure(range(6), range(6))

        result = execute(qc, self.backend, shots=1024).result()
        counts = result.get_counts()
        quantum_metric = int(min(counts, key=counts.get), 2)

        # Normalize quantum metric
        normalized_metric = quantum_metric / len(self.stops)
        return normalized_metric
      except Exception as e:
        print(f"Error in phase estimation: {e}")
        return float('inf')  # High metric value in case of an error


    def modified_dijkstra(self, source, destination, start_time):
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                if quantum_metric == float('inf'):
                    continue

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None


In [ ]:
# Quantum A* Implementation
class QuantumAStar:
    def __init__(self, nodes, adjacency_list):
        self.nodes = nodes
        self.graph = adjacency_list

def quantum_heuristic(self, current, destination):
    try:
        qc = QuantumCircuit(6, 6)
        current_coords = self.nodes[current]['coords']
        dest_coords = self.nodes[destination]['coords']
        distance = ((current_coords[0] - dest_coords[0]) ** 2 +
                    (current_coords[1] - dest_coords[1]) ** 2) ** 0.5
        for i in range(6):
            qc.p(distance / (2 ** i), i)  # Use phase (p) gate
        qc.measure(range(6), range(6))
        backend = Aer.get_backend('qasm_simulator')
        result = execute(qc, backend, shots=1024).result()
        counts = result.get_counts()
        heuristic = int(min(counts, key=counts.get), 2)

        # Normalize heuristic to align with distance and time scales
        normalized_heuristic = heuristic / len(self.nodes)
        return normalized_heuristic
    except Exception as e:
        print(f"Error in quantum heuristic: {e}")
        return float('inf')  # High heuristic value in case of an error


    def find_optimized_route(self, source, destination):
        queue = [(0, source, [source])]
        visited = set()

        while queue:
            f_cost, current, path = heapq.heappop(queue)

            if current in visited:
                continue
            visited.add(current)

            if current == destination:
                return f_cost, path

            for neighbor, data in self.graph[current].items():
                if neighbor not in visited:
                    distance = data['distance']
                    time = data['time']
                    heuristic = self.quantum_heuristic(neighbor, destination)
                    new_f = f_cost + distance + 0.1 * time + heuristic
                    heapq.heappush(queue, (new_f, neighbor, path + [neighbor]))

        return float('inf'), []

In [ ]:
# QAOA-Based Optimization
def create_qubo(graph, source, destination):
    qubo = QuadraticProgram()
    for node in graph:
        qubo.binary_var(f"x_{node}")

    quadratic_terms = {}
    for node, neighbors in graph.items():
        for neighbor, details in neighbors.items():
            quadratic_terms[(f"x_{node}", f"x_{neighbor}")] = details['distance']

    qubo.minimize(quadratic=quadratic_terms)
    qubo.linear_constraint({f"x_{source}": 1}, "==", 1, "source_constraint")
    qubo.linear_constraint({f"x_{destination}": 1}, "==", 1, "destination_constraint")

    return qubo


def solve_qubo_classically(qubo, graph):
    converter = QuadraticProgramToQubo()
    qubo = converter.convert(qubo)

    num_vars = len(qubo.variables)
    qubo_matrix = np.zeros((num_vars, num_vars))

    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        qubo_matrix[i, j] = value
        qubo_matrix[j, i] = value

    def objective(x):
        return x @ qubo_matrix @ x.T

    initial_guess = np.random.choice([0, 1], size=num_vars)
    result = minimize(objective, initial_guess, bounds=[(0, 1)] * num_vars, method="L-BFGS-B")
    solution = np.round(result.x).astype(int)

    total_cost = 0
    path_edges = []
    for (i, j), value in qubo.objective.quadratic.to_dict().items():
        if solution[i] > 0.5 and solution[j] > 0.5:
            node_i = qubo.variables[i].name.split("_")[1]
            node_j = qubo.variables[j].name.split("_")[1]
            if node_j in graph[node_i]:  # Ensure the edge exists in the graph
                total_cost += graph[node_i][node_j]['distance']
                path_edges.append((node_i, node_j))

    return solution, total_cost, path_edges

In [ ]:
# Simulation Setup
num_nodes = 50
max_edges = 3
max_distance = 50
max_time = 30
dynamic_graph = DynamicGraph(num_nodes, max_edges, max_distance, max_time)

# Print Initial Graph
dynamic_graph.print_graph()

# Quantum Modified Dijkstra
qmd = QuantumModifiedDijkstra(dynamic_graph.nodes, dynamic_graph.graph)
source = "Node_1"
destination = f"Node_{num_nodes}"
start_time = datetime.now()
cost_dijkstra, path_dijkstra, end_time_dijkstra = qmd.modified_dijkstra(source, destination, start_time)

print("\nQuantum Modified Dijkstra Results:")
print(f"Cost: {cost_dijkstra}")
print(f"Path: {path_dijkstra}")
print(f"End Time: {end_time_dijkstra}")




Graph Nodes and Edges:
Node: Node_1, Coordinates: (82.04776758748194, 19.491320774398126)
  -> Node_33: Distance=2.28, Time=20.19
  -> Node_25: Distance=45.27, Time=19.53
  -> Node_35: Distance=22.77, Time=20.93
Node: Node_2, Coordinates: (49.088763311887675, 41.50648248789178)
  -> Node_39: Distance=33.15, Time=24.01
  -> Node_48: Distance=30.63, Time=2.04
Node: Node_3, Coordinates: (96.75651177879229, 89.27210537971393)
  -> Node_45: Distance=46.79, Time=1.52
  -> Node_22: Distance=21.59, Time=10.64
  -> Node_25: Distance=11.97, Time=13.40
Node: Node_4, Coordinates: (0.8772423812809405, 52.57474328897672)
  -> Node_42: Distance=40.51, Time=17.69
Node: Node_5, Coordinates: (93.19382905711973, 55.91167167026024)
  -> Node_6: Distance=16.70, Time=11.35
  -> Node_7: Distance=15.47, Time=25.80
Node: Node_6, Coordinates: (31.561248225401194, 23.816993448630985)
  -> Node_28: Distance=6.19, Time=14.68
Node: Node_7, Coordinates: (59.365739911637704, 54.88802897358938)
  -> Node_21: Distance

In [ ]:
quantum_astar = QuantumAStar(dynamic_graph.nodes, dynamic_graph.graph)
# Run Quantum A*
cost_astar, path_astar = quantum_astar.find_optimized_route(source, destination)

# Print Quantum A* Results
print("\nQuantum A* Results:")
print(f"Cost: {cost_astar}")
print(f"Path: {path_astar}")


Quantum A* Results:
Cost: 69.52677695216924
Path: ['Node_1', 'Node_12', 'Node_35', 'Node_50']


In [ ]:
# Dynamic Graph Setup
class DynamicGraph:
    def __init__(self, num_nodes, max_edges, max_distance, max_time):
        self.nodes = {
            f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))}
            for i in range(1, num_nodes + 1)
        }
        self.graph = {
            node: {
                neighbor: {
                    "distance": random.uniform(1, max_distance),
                    "time": random.uniform(1, max_time),
                }
                for neighbor in random.sample(list(self.nodes.keys()), random.randint(1, max_edges))
                if neighbor != node
            }
            for node in self.nodes
        }

    def print_graph(self):
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")

In [ ]:
import random
import heapq
from datetime import datetime, timedelta
from qiskit import QuantumCircuit, Aer, execute

# Quantum A* Implementation
class QuantumAStar:
    def __init__(self, nodes, adjacency_list):
        self.nodes = nodes
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def quantum_heuristic(self, current, destination):
        """
        Heuristic calculation with Quantum Phase Estimation.
        """
        try:
            qc = QuantumCircuit(6, 6)
            current_coords = self.nodes[current]['coords']
            dest_coords = self.nodes[destination]['coords']
            distance = ((current_coords[0] - dest_coords[0]) ** 2 +
                        (current_coords[1] - dest_coords[1]) ** 2) ** 0.5
            for i in range(6):
                qc.p(distance / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))
            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            heuristic = int(min(counts, key=counts.get), 2)

            # Normalize heuristic to align with distance and time scales
            normalized_heuristic = heuristic / len(self.nodes)
            return normalized_heuristic
        except Exception as e:
            print(f"Error in quantum heuristic: {e}")
            return float('inf')

    def find_optimized_route(self, source, destination):
        queue = [(0, source, [source])]
        visited = set()

        while queue:
            f_cost, current, path = heapq.heappop(queue)

            if current in visited:
                continue
            visited.add(current)

            if current == destination:
                return f_cost, path

            for neighbor, data in self.graph[current].items():
                if neighbor not in visited:
                    distance = data['distance']
                    time = data['time']
                    heuristic = self.quantum_heuristic(neighbor, destination)
                    new_f = f_cost + distance + 0.1 * time + heuristic
                    heapq.heappush(queue, (new_f, neighbor, path + [neighbor]))

        return float('inf'), []

# Quantum Modified Dijkstra Implementation
class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
        """
        Use Quantum Phase Estimation (QPE) to encode the cost.
        """
        try:
            qc = QuantumCircuit(6, 6)
            qc.h(range(6))
            composite_metric = distance + 0.1 * time

            for i in range(6):
                qc.p(composite_metric / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))

            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            quantum_metric = int(min(counts, key=counts.get), 2)

            # Normalize quantum metric
            normalized_metric = quantum_metric / len(self.stops)
            return normalized_metric
        except Exception as e:
            print(f"Error in phase estimation: {e}")
            return float('inf')

    def modified_dijkstra(self, source, destination, start_time):
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                if quantum_metric == float('inf'):
                    continue

                new_metric = composite_metric + quantum_metric
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None



# Simulation
num_nodes = 10
max_edges = 3
max_distance = 50
max_time = 30
dynamic_graph = DynamicGraph(num_nodes, max_edges, max_distance, max_time)

# Print Initial Graph
print("Initial Graph:")
dynamic_graph.print_graph()

# Initialize Algorithms
quantum_astar = QuantumAStar(dynamic_graph.nodes, dynamic_graph.graph)
quantum_dijkstra = QuantumModifiedDijkstra(dynamic_graph.nodes, dynamic_graph.graph)

# Define Source and Destination
source = "Node_1"
destination = f"Node_{num_nodes}"
start_time = datetime.now()

# Run Quantum A*
cost_astar, path_astar = quantum_astar.find_optimized_route(source, destination)
print("\nQuantum A* Results:")
print(f"Cost: {cost_astar:.2f}, Path: {path_astar}")

# Run Quantum Modified Dijkstra
cost_dijkstra, path_dijkstra, end_time = quantum_dijkstra.modified_dijkstra(source, destination, start_time)
print("\nQuantum Modified Dijkstra Results:")
print(f"Cost: {cost_dijkstra:.2f}, Path: {path_dijkstra}, End Time: {end_time}")


Initial Graph:

Graph Nodes and Edges:
Node: Node_1, Coordinates: (28.066854592604184, 69.5651238229943)
  -> Node_4: Distance=20.72, Time=26.81
Node: Node_2, Coordinates: (78.69622544782897, 13.338682331676232)
  -> Node_3: Distance=10.79, Time=16.05
  -> Node_4: Distance=9.50, Time=8.02
Node: Node_3, Coordinates: (72.66834220913168, 9.83146329218596)
  -> Node_6: Distance=46.93, Time=20.96
Node: Node_4, Coordinates: (78.47793654347768, 56.28324231181847)
  -> Node_10: Distance=46.59, Time=20.18
  -> Node_3: Distance=9.28, Time=15.63
  -> Node_2: Distance=20.22, Time=8.43
Node: Node_5, Coordinates: (75.12355820676765, 82.54666624476702)
  -> Node_9: Distance=19.48, Time=23.55
Node: Node_6, Coordinates: (87.39779222851737, 0.7420078456545687)
  -> Node_3: Distance=14.85, Time=19.88
  -> Node_9: Distance=45.50, Time=29.92
Node: Node_7, Coordinates: (51.31045941914911, 4.619913710487089)
  -> Node_10: Distance=48.70, Time=3.33
  -> Node_2: Distance=38.03, Time=14.81
Node: Node_8, Coordin

In [ ]:
import random
import heapq
from datetime import datetime, timedelta
from qiskit import QuantumCircuit, Aer, execute

# Helper function for consistent cost calculation
def calculate_cost(distance, time, quantum_value=0):
    """Calculate the total cost including distance, time, and any quantum value."""
    return distance + 0.1 * time + quantum_value

# Dynamic Graph Simulation
class DynamicGraph:
    def __init__(self, num_nodes, max_edges, max_distance, max_time):
        self.nodes = {f"Node_{i}": {"coords": (random.uniform(0, 100), random.uniform(0, 100))} for i in range(1, num_nodes + 1)}
        self.graph = {node: {
            neighbor: {"distance": random.uniform(1, max_distance), "time": random.uniform(1, max_time)}
            for neighbor in random.sample(list(self.nodes.keys()), random.randint(1, max_edges)) if neighbor != node
        } for node in self.nodes}

    def update_graph(self):
        """Simulate real-time updates to the graph."""
        for node, edges in self.graph.items():
            for neighbor in list(edges.keys()):
                if random.random() < 0.1:  # 10% chance to remove an edge
                    del self.graph[node][neighbor]
                else:  # Adjust weights
                    self.graph[node][neighbor]['distance'] *= random.uniform(0.9, 1.1)
                    self.graph[node][neighbor]['time'] *= random.uniform(0.9, 1.1)

    def print_graph(self):
        """Print the graph nodes and edges."""
        print("\nGraph Nodes and Edges:")
        for node, edges in self.graph.items():
            print(f"Node: {node}, Coordinates: {self.nodes[node]['coords']}")
            for neighbor, details in edges.items():
                print(f"  -> {neighbor}: Distance={details['distance']:.2f}, Time={details['time']:.2f}")

# Quantum A* Implementation
class QuantumAStar:
    def __init__(self, nodes, adjacency_list):
        self.nodes = nodes
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def quantum_heuristic(self, current, destination):
        """Quantum heuristic calculation using Phase Estimation."""
        try:
            qc = QuantumCircuit(6, 6)
            current_coords = self.nodes[current]['coords']
            dest_coords = self.nodes[destination]['coords']
            distance = ((current_coords[0] - dest_coords[0]) ** 2 +
                        (current_coords[1] - dest_coords[1]) ** 2) ** 0.5
            for i in range(6):
                qc.p(distance / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))
            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            quantum_value = int(min(counts, key=counts.get), 2) / len(self.nodes)  # Normalize
            return quantum_value
        except Exception as e:
            print(f"Error in quantum heuristic: {e}")
            return 0

    def find_optimized_route(self, source, destination):
        """Find the optimized route from source to destination."""
        queue = [(0, source, [source])]
        visited = set()

        while queue:
            f_cost, current, path = heapq.heappop(queue)

            if current in visited:
                continue
            visited.add(current)

            if current == destination:
                return f_cost, path

            for neighbor, data in self.graph[current].items():
                if neighbor not in visited:
                    distance = data['distance']
                    time = data['time']
                    heuristic = self.quantum_heuristic(neighbor, destination)
                    new_f = calculate_cost(distance, time, heuristic)
                    heapq.heappush(queue, (new_f, neighbor, path + [neighbor]))

        return float('inf'), []

# Quantum Modified Dijkstra Implementation
class QuantumModifiedDijkstra:
    def __init__(self, stops, adjacency_list):
        self.stops = stops
        self.graph = adjacency_list
        self.backend = Aer.get_backend('qasm_simulator')

    def phase_estimation(self, distance, time):
        """Use Quantum Phase Estimation (QPE) to encode the cost."""
        try:
            qc = QuantumCircuit(6, 6)
            qc.h(range(6))
            composite_metric = distance + 0.1 * time

            for i in range(6):
                qc.p(composite_metric / (2 ** i), i)  # Use phase (p) gate
            qc.measure(range(6), range(6))

            result = execute(qc, self.backend, shots=1024).result()
            counts = result.get_counts()
            quantum_metric = int(min(counts, key=counts.get), 2) / len(self.stops)  # Normalize
            return quantum_metric
        except Exception as e:
            print(f"Error in phase estimation: {e}")
            return 0

    def modified_dijkstra(self, source, destination, start_time):
        """Find the optimal route using Modified Dijkstra's algorithm."""
        queue = [(0, start_time, source, [source])]
        visited = set()

        while queue:
            composite_metric, current_time, current, path = heapq.heappop(queue)

            if current == destination:
                return composite_metric, path, current_time

            state = (current, tuple(path))
            if state in visited:
                continue
            visited.add(state)

            for neighbor, details in self.graph[current].items():
                distance = details['distance']
                time = details['time']

                quantum_metric = self.phase_estimation(distance, time)
                new_metric = calculate_cost(distance, time, quantum_metric)
                new_time = current_time + timedelta(minutes=time)
                new_path = path + [neighbor]
                heapq.heappush(queue, (new_metric, new_time, neighbor, new_path))

        return float('inf'), [], None

# Simulation Setup
num_nodes = 90
max_edges = 3
max_distance = 50
max_time = 30

dynamic_graph = DynamicGraph(num_nodes, max_edges, max_distance, max_time)

# Print Initial Graph
print("Initial Graph:")
dynamic_graph.print_graph()

# Initialize Algorithms
quantum_astar = QuantumAStar(dynamic_graph.nodes, dynamic_graph.graph)
quantum_dijkstra = QuantumModifiedDijkstra(dynamic_graph.nodes, dynamic_graph.graph)

# Define Source and Destination
source = "Node_4"
destination = f"Node_{70}"
start_time = datetime.now()

# Run Quantum A*
cost_astar, path_astar = quantum_astar.find_optimized_route(source, destination)
print("\nQuantum A* Results:")
print(f"Cost: {cost_astar:.2f}, Path: {path_astar}")

# Run Quantum Modified Dijkstra
cost_dijkstra, path_dijkstra, end_time = quantum_dijkstra.modified_dijkstra(source, destination, start_time)
print("\nQuantum Modified Dijkstra Results:")
print(f"Cost: {cost_dijkstra:.2f}, Path: {path_dijkstra}, End Time: {end_time}")


Initial Graph:

Graph Nodes and Edges:
Node: Node_1, Coordinates: (46.17894516872107, 70.24496486769935)
  -> Node_25: Distance=23.51, Time=17.77
Node: Node_2, Coordinates: (53.80053145089979, 53.98453345163484)
  -> Node_63: Distance=4.93, Time=1.89
  -> Node_26: Distance=8.81, Time=13.63
  -> Node_1: Distance=33.96, Time=16.69
Node: Node_3, Coordinates: (6.0553735743743236, 35.47489871734286)
  -> Node_23: Distance=12.07, Time=22.94
  -> Node_5: Distance=2.53, Time=23.88
  -> Node_29: Distance=27.45, Time=20.12
Node: Node_4, Coordinates: (82.91407557119614, 96.62582586194124)
  -> Node_14: Distance=12.07, Time=24.49
  -> Node_10: Distance=17.32, Time=5.64
Node: Node_5, Coordinates: (15.947892948481869, 62.27555050104105)
  -> Node_41: Distance=32.49, Time=12.90
Node: Node_6, Coordinates: (89.8294217903015, 89.88868794353525)
  -> Node_41: Distance=10.36, Time=2.00
Node: Node_7, Coordinates: (41.544000604559585, 42.58508645302028)
  -> Node_60: Distance=33.91, Time=18.01
  -> Node_39:

In [ ]:
import heapq
import numpy as np

class ImprovedSimulatedQAOA:
    def __init__(self, graph, nodes):
        """
        Initialize the simulated QAOA with a graph and node details.
        :param graph: Adjacency list representation of the graph.
        :param nodes: Dictionary of node details with coordinates.
        """
        self.graph = graph
        self.nodes = nodes

    def heuristic(self, current, destination):
        """
        Simulated heuristic using normalized Euclidean distance.
        :param current: Current node.
        :param destination: Target node.
        :return: Normalized heuristic value.
        """
        current_coords = self.nodes[current]['coords']
        dest_coords = self.nodes[destination]['coords']
        distance = ((current_coords[0] - dest_coords[0]) ** 2 +
                    (current_coords[1] - dest_coords[1]) ** 2) ** 0.5
        return distance / len(self.nodes)  # Normalize by the number of nodes

    def solve(self, source, destination):
        """
        Simulate a QAOA-based pathfinding from source to destination.
        :param source: Starting node.
        :param destination: Target node.
        :return: Total cost and path.
        """
        # Priority queue to simulate optimization process
        queue = [(0, source, [source])]  # (cost, current_node, path)
        visited = set()

        while queue:
            cost, current, path = heapq.heappop(queue)

            if current in visited:
                continue
            visited.add(current)

            if current == destination:
                # Clean path to ensure no duplicates
                cleaned_path = []
                for node in path:
                    if not cleaned_path or cleaned_path[-1] != node:
                        cleaned_path.append(node)
                return cost, cleaned_path

            for neighbor, data in self.graph[current].items():
                if neighbor not in visited:
                    # Simulate QAOA heuristic
                    heuristic = self.heuristic(neighbor, destination)
                    new_cost = cost + data['distance'] + 0.1 * data['time'] + heuristic
                    heapq.heappush(queue, (new_cost, neighbor, path + [neighbor]))

        return float('inf'), []



# Simulated QAOA Execution with the Generated Graph
simulated_qaoa = ImprovedSimulatedQAOA(dynamic_graph.graph, dynamic_graph.nodes)

# Solve using Simulated QAOA
qaoa_cost, qaoa_path = simulated_qaoa.solve(source, destination)

# Print Results
print("\nSimulated QAOA Results:")
print(f"Cost: {qaoa_cost:.2f}, Path: {qaoa_path}")



Simulated QAOA Results:
Cost: 195.05, Path: ['Node_4', 'Node_53', 'Node_43', 'Node_36', 'Node_18', 'Node_41', 'Node_45', 'Node_37', 'Node_70']
